## 第一章 起步

- 简介
- 下载与安装
- 基本使用

In [42]:
import tensorflow as tf
import numpy as np

# 使用 NumPy 生成假数据(phony data), 总共 100 个点.
x_data = np.float32(np.random.rand(2, 100)) # 随机输入
y_data = np.dot([0.100, 0.200], x_data) + 0.300

# 构造一个线性模型
#
b = tf.Variable(tf.zeros([1]))
W = tf.Variable(tf.random_uniform([1, 2], -1.0, 1.0))
y = tf.matmul(W, x_data) + b

# 最小化方差
loss = tf.reduce_mean(tf.square(y - y_data))
optimizer = tf.train.GradientDescentOptimizer(0.5)
train = optimizer.minimize(loss)

# loss，optimizer，train定义好了就会自己训练吗

# 初始化变量
init = tf.initialize_all_variables()
# 为什么要初始化变量，有什么变量需要初始化，好像是初始化Variable

# 启动图 (graph)
sess = tf.Session()
sess.run(init)

# 拟合平面
for step in range(0, 201):
    sess.run(train)
    
    if step % 20 == 0:
        print(step, sess.run(W), sess.run(b))
    # 得到最佳拟合结果 W: [[0.100 0.200]], b: [0.300]


0 [[-0.44216108  0.35331267]] [0.9826096]
20 [[-0.03490521  0.17342135]] [0.37992156]
40 [[0.07401735 0.18765137]] [0.31919065]
60 [[0.09496708 0.19532934]] [0.30491433]
80 [[0.09901606 0.1983728 ]] [0.30133587]
100 [[0.09980481 0.19945548]] [0.30038148]
120 [[0.09996038 0.19982165]] [0.30011305]
140 [[0.09999168 0.1999423 ]] [0.30003437]
160 [[0.09999818 0.19998148]] [0.30001062]
180 [[0.09999956 0.19999407]] [0.30000332]
200 [[0.09999989 0.19999813]] [0.30000106]


In [2]:
import tensorflow as tf
hello = tf.constant('Hello, TensorFlow!')
sess = tf.Session()
print(sess.run(hello))

a = tf.constant(10)
b = tf.constant(32)
print(sess.run(a+b))


b'Hello, TensorFlow!'
42


### 基本使用
使用图Graph来表示计算任务
- 使用图（graph）来表示计算任务
- 在被称为（会话session）的上下文（context）中执行图
- 使用tensor表示数据
- 通过变量（Variable）维护状态
- 使用feed和fetch可以为任意操作（arbitrary operation）赋值或者从其中获取数据

### 综述
Tensorflow是一个编程系统，使用图来表示计算任务，图中的节点被称至为op(operation的缩写)，一个op获得0个或多个Tensor，每个Tensor是一个类型化的多维数组，例如，可以将一小组图像集表示为一个四维浮点数数组[batch, height, width, channels]

一个Tensor图描述了计算的过程，为了进行计算，图必须在会话里被启动，会话将图的op分发到CPU或GPU之类的设备上，同时提供op的方法，这些方法执行后，将产生的tensor返回，在python语言中，返回的tensor是numpy ndarray对象。

### 计算图
TensorFlow程序通常被组织称一个构建阶段和一个执行阶段。
通常在构建阶段创建一个图来表示和训练神经网络，然后在执行阶段反复执行图中的训练Op。
TensorFlow支持C，C++，Python编程语言，目前Tensorflow的python库更加易用。

三种语言的会话库是一致的。

#### 构建图
创建源op（source op），源op不需要任何输入，例如常量（Constant），源op的输出被传递给其他op做计算。

Python库中，op构造器的返回值代表被构造出的op的输出，这些返回值可以传递给其他op构造器作为输入。

In [3]:
import tensorflow as tf


# 创建一个常量 op, 产生一个 1x2 矩阵. 这个 op 被作为一个节点
# 加到默认图中.
#
# 构造器的返回值代表该常量 op 的返回值.
matrix1 = tf.constant([[3., 3.]])


# 创建另外一个常量 op, 产生一个 1 x 2 矩阵.
matrix2 = tf.constant([[2.],[2.]])


# 创建一个矩阵乘法 matmul op , 把 'matrix1' 和 'matrix2' 作为输入.
# 返回值 'product' 代表矩阵乘法的结果.
product = tf.matmul(matrix1, matrix2)


(1, 2)


In [4]:
print(product)

Tensor("MatMul:0", shape=(1, 1), dtype=float32)


In [5]:
# 构造阶段完成后，才能启动图，启动图的第一步是创建一个Session对象，如果无任何创建参数，会话构造器将启动默认图。

sess = tf.Session()

result = sess.run(product)
print(result)
# 计算图的输出是什么，假如有多个输出怎么办

sess.close()

[[12.]]


In [6]:
## Session对象在使用完后需要关闭以释放资源，除了显示调用close外，也可以使用 with 代码块，自动完成关闭动作

with tf.Session() as sess:
    result = sess.run([product])
    print(result)

[array([[12.]], dtype=float32)]


In [9]:
with tf.Session() as sess:
    with tf.device("/gpu:1"):
        matrix1 = tf.constant([[3., 3.]])
        matrix2 = tf.constant([[2.], [2.]])
        product = tf.matmul(matrix1, matrix2)
        print(sess.run([product]))

[array([[12.]], dtype=float32)]


### 交互式使用
文档中的python示例使用一个会话Session来启动图，并调用Session.run()方法执行操作。
为了便于使用Ipython之类的Python交互环境使用，可以使用InteractiveSession代替Session类，使用Tensor.eval()和Operation.run()方法代替Session.run()，这样可以避免使用一个变量来持有会话。

In [11]:
import tensorflow as tf
sess = tf.InteractiveSession()

x = tf.Variable([1.0, 2.0])
a = tf.constant([3.0, 3.0])

# 使用初始化器 initialize op的run方法初始化 x

x.initializer.run()

# 增加一个减法 sub op，从'x'减轻a，运行减法op，输出结果

sub = tf.subtract(x, a)
print(sub.eval())

[-2. -1.]


/data/miniconda3/envs/tf1.14/lib/python3.6/site-packages/tensorflow/python/client/session.py:1735: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


## Tensor

TensorFlow程序使用tensor数据结构来代表所有的数据，计算图中，操作间传递的数据都是tensor，你可以把TensorFlow tensor看作是一个n维的数组或列表。一个tensor包含一个静态类型rank，一个shape。

In [53]:
## 变量
# 变量维护图执行过程中的状态信息。下面的例子演示了如何使用变量实现一个简单的计数器，

## 创建一个变量，初始化为标量0

state = tf.Variable(0, name="counter")

# 创建一个Op，其作用是使state增加1

one = tf.constant(1)

new_value = tf.add(state, one)
update = tf.assign(state, new_value)

# 启动图后，变量必须先经过”初始化“（init）op初始化

init_op = tf.initialize_all_variables()

# 启动图，运行op

with tf.Session() as sess:
    # 运行 init op
    sess.run(init_op)
    # 打印 'state'的初始值
    # print(sess.run(state))
    
    # 运行op，更新state，并打印state
    for _ in range(3):
        print(sess.run(state))
        print(sess.run(new_value))
        print(sess.run(update))
        # print(sess.run(state))
        
# tensorflow的执行过程到底是怎么样的
        
# 代码中assign()操作是图所描绘的表达式的一部分，正如add()操作一样，所以在调用run()执行表达式之前，它并不会真正执行赋值操作

# 通常会将一个统计模型中的参数表示为一组变量，例如，你可以将一个神经网络的权重作为某个变量存储在一个tensor中，在训练过程中，通过重复运行训练图，更新这个tensor。

0
1
1
1
2
2
2
3
3


In [17]:
## Fetch

# 为了取回操作的输出内容，可以在使用Session对象的run()调用执行图时，传入一些tensor，这些tensor会帮助你取回结果，在之前的例子里，我们只取回了单个节点state，但你也可以取回多个tensor

input1 = tf.constant(3.0)
input2 = tf.constant(2.0)
input3 = tf.constant(5.0)

intermed = tf.add(input2, input3)
mul = tf.multiply(input1, intermed)

with tf.Session() as sess:
    # 在一次 run 的过程，我们可以同时获取多个值
    
    result = sess.run([mul, intermed])
    print(result)

[21.0, 7.0]


In [21]:
## Feed
# 临时替代Tensor

# 用placeholder占位，之后通过feed喂入数据
input1 = tf.placeholder(tf.float32)
input2 = tf.placeholder(tf.float32)
output = tf.multiply(input1, input2)

with tf.Session() as sess:
    print(sess.run([output], feed_dict={input1:[7.], input2:[2.]}))
    
# 对于大规模样例，需要正确提供feed，及placeholder()操作

[array([14.], dtype=float32)]


In [35]:
x = [[1,2,3],
     [4,5,6],
     [7,8,9]]
# 2 3
(0,0), (0,1), (0,2)
(1,0), (1,1), (1,2)

y = tf.cast(x, tf.float32)

mean_all = tf.reduce_mean(y, axis=1, keep_dims=True)
square_all = tf.square(y)

In [37]:
with tf.Session() as sess:
    a, b = sess.run([mean_all, square_all])
    print(a)
    print(b)

[[2.]
 [5.]
 [8.]]
[[ 1.  4.  9.]
 [16. 25. 36.]
 [49. 64. 81.]]
